In [1]:
import pandas as pd
import re
import random
import numpy as np

In [ ]:
def get_df_norm_header_index(path):
    lst_header = ['num_exp', 'dia', 'hora', 'calle', 'num', 'distrito', 'tipo_accidente', 'meteo', 'vehiculo', 'persona', 'edad', 'sexo', 'lesividad' ]
    df = pd.read_csv(path, encoding='latin_1',sep = ';', names=lst_header)
    df.drop(0, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

bici = get_df_norm_header_index('bicicletas/AccidentesBicicletas_2019.csv')


In [2]:
df = pd.read_csv('bicicletas/AccidentesBicicletas_2018.csv', encoding='latin_1',sep = ';')

In [3]:
df

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,...,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,* Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad
0,01/01/2018,DE 13:00 A 13:59,LUNES,ARGANZUELA,CALLE DE ANCORA - PASEO DE LAS DELICIAS ...,0,2018/25,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,BICICLETA,CONDUCTOR,HOMBRE,HL,DE 30 A 34 ANOS
1,01/01/2018,DE 15:00 A 15:59,LUNES,CENTRO,CALLE DE ALCALA NUM ...,44,2018/76,NO,NO,NO,...,NO,NO,SI,2,ATROPELLO,BICICLETA,CONDUCTOR,HOMBRE,HL,DE 25 A 29 AÑOS
2,02/01/2018,DE 12:00 A 12:59,MARTES,SALAMANCA,CALLE DE SERRANO - CALLE DEL CONDE DE ARANDA ...,0,2018/30,NO,NO,NO,...,NO,NO,SI,2,ATROPELLO,BICICLETA,CONDUCTOR,HOMBRE,HL,DE 40 A 44 AÑOS
3,03/01/2018,DE 13:00 A 13:59,MIERCOLES,SALAMANCA,PLAZA DE LA INDEPENDENCIA NUM ...,5,2018/94,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,BICICLETA,CONDUCTOR,HOMBRE,HL,DE 50 A 54 AÑOS
4,03/01/2018,DE 15:00 A 15:59,MIERCOLES,CIUDAD LINEAL,CALLE DE RAMIREZ DE ARELLANO NUM ...,35,2018/361,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,BICICLETA,CONDUCTOR,HOMBRE,HL,DE 25 A 29 AÑOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,25/12/2018,DE 12:00 A 12:59,MARTES,FUENCARRAL-EL PARDO,CALLE DE BRAOJOS NUM ...,10,2018/16468,NO,NO,NO,...,NO,NO,SI,1,CAÍDA BICICLETA,BICICLETA,CONDUCTOR,HOMBRE,HL,DE 25 A 29 AÑOS
696,28/12/2018,DE 9:00 A 9:59,VIERNES,CHAMARTIN,PASEO DE LA CASTELLANA - PASEO DE LA HABANA ...,0,2018/16651,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,BICICLETA,CONDUCTOR,HOMBRE,HL,DE 25 A 29 AÑOS
697,28/12/2018,DE 14:00 A 14:59,VIERNES,CENTRO,CALLE DE GENOVA NUM ...,2,2018/16626,NO,NO,NO,...,NO,NO,SI,2,COLISIÓN DOBLE,BICICLETA,CONDUCTOR,HOMBRE,HL,DE 18 A 20 AÑOS
698,29/12/2018,DE 10:00 A 10:59,SABADO,TETUAN,CALLE DE BRAVO MURILLO NUM ...,105,2019/57,NO,NO,NO,...,NO,NO,SI,1,COLISIÓN DOBLE,BICICLETA,CONDUCTOR,HOMBRE,HL,DE 55 A 59 AÑOS


In [3]:
bici.dtypes

num_exp           object
dia               object
hora              object
calle             object
num               object
distrito          object
tipo_accidente    object
meteo             object
vehiculo          object
persona           object
edad              object
sexo              object
lesividad         object
dtype: object

In [4]:
def create_date(df):
    df['fecha'] = df.dia+' '+df.hora
    df['fecha'] = pd.to_datetime(df['fecha'], dayfirst=True)
    df['fecha'] = df['fecha'].dt.round('H')
    df.drop(['dia', 'hora'], axis=1, inplace = True)
    return df

bici = create_date(bici)

In [5]:
def create_dir(df):
    df.num.apply(lambda x: x.strip())
    df.loc[(df.num == '-'), 'num']=''
    df['direccion'] = df.calle+' '+df.num
    df.direccion.apply(lambda x: x.strip())
    df.drop(['calle', 'num'], axis=1, inplace = True)
    return df

bici = create_dir(bici)

In [6]:
bici.head()

,num_exp,distrito,tipo_accidente,meteo,vehiculo,persona,edad,sexo,lesividad,fecha,direccion
0,2019S000659,SALAMANCA,Alcance,Despejado,Bicicleta,Conductor,DE 25 A 29 AÑOS,Hombre,01,2019-01-01 14:00:00,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ
1,2019S000036,HORTALEZA,Colisión fronto-lateral,Despejado,Bicicleta,Conductor,DE 70 A 74 AÑOS,Hombre,03,2019-01-02 21:00:00,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES
2,2019S000133,VILLA DE VALLECAS,Alcance,Se desconoce,Bicicleta,Conductor,DE 15 A 17 AÑOS,Hombre,07,2019-01-03 14:00:00,CALL. FELIPE ALVAREZ 10
3,2019S000132,VILLA DE VALLECAS,Alcance,Despejado,Bicicleta,Conductor,DE 18 A 20 AÑOS,Hombre,07,2019-01-03 13:00:00,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA
4,2019S000132,VILLA DE VALLECAS,Alcance,Despejado,Bicicleta,Conductor,DE 21 A 24 AÑOS,Hombre,14,2019-01-03 13:00:00,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA


In [7]:
bici.tipo_accidente.isna().sum()

0

In [ ]:
def fill_drop(df):
    df.lesividad.fillna('14', inplace=True)
    df.meteo.fillna('Se desconoce', inplace=True)
    df.drop(['sexo', 'vehiculo', 'persona', 'edad'], axis=1, inplace=True)
    return df

bici = fill_drop(bici)

In [8]:
def change_accident(values):
    dict_accidente = {
        r'Caída':'Caida',
        r'Colisión':'Colision',
        r'Alcance':'Colision',
        r'Atropello':'Imprudencia',
        r'Choque':'Imprudencia'}
    for k, v in dict_accidente.items():
        if re.match(k, values):
            return v
    return 'Otro'

bici.tipo_accidente = bici.tipo_accidente.apply(lambda x: change_accident(x))

In [9]:
bici.tipo_accidente.value_counts(dropna= False)

Colision       388
Caida          230
Imprudencia    141
Otro            20
Name: tipo_accidente, dtype: int64

In [10]:
bici.lesividad.value_counts(dropna=False)

07     275
02     120
14      98
06      83
NaN     80
01      66
05      35
03      21
04       1
Name: lesividad, dtype: int64

In [12]:
def change_injury(values):
    dict_injury = {
        '01':'Moderada','02':'Moderada',
        '03':'Grave',
        '04':'Fallecido',
        '05':'Leve','06':'Leve','07':'Leve','14':'Leve'}
    return dict_injury[values]


bici.lesividad = bici.lesividad.apply(lambda x: change_injury(str(x)))

In [13]:
bici.head()

,num_exp,distrito,tipo_accidente,meteo,vehiculo,persona,edad,sexo,lesividad,fecha,direccion
0,2019S000659,SALAMANCA,Colision,Despejado,Bicicleta,Conductor,DE 25 A 29 AÑOS,Hombre,Moderada,2019-01-01 14:00:00,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ
1,2019S000036,HORTALEZA,Colision,Despejado,Bicicleta,Conductor,DE 70 A 74 AÑOS,Hombre,Grave,2019-01-02 21:00:00,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES
2,2019S000133,VILLA DE VALLECAS,Colision,Se desconoce,Bicicleta,Conductor,DE 15 A 17 AÑOS,Hombre,Leve,2019-01-03 14:00:00,CALL. FELIPE ALVAREZ 10
3,2019S000132,VILLA DE VALLECAS,Colision,Despejado,Bicicleta,Conductor,DE 18 A 20 AÑOS,Hombre,Leve,2019-01-03 13:00:00,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA
4,2019S000132,VILLA DE VALLECAS,Colision,Despejado,Bicicleta,Conductor,DE 21 A 24 AÑOS,Hombre,Leve,2019-01-03 13:00:00,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA


In [19]:
bici.head()

,num_exp,distrito,tipo_accidente,meteo,lesividad,fecha,direccion
0,2019S000659,SALAMANCA,Colision,Despejado,Moderada,2019-01-01 14:00:00,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ
1,2019S000036,HORTALEZA,Colision,Despejado,Grave,2019-01-02 21:00:00,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES
2,2019S000133,VILLA DE VALLECAS,Colision,Se desconoce,Leve,2019-01-03 14:00:00,CALL. FELIPE ALVAREZ 10
3,2019S000132,VILLA DE VALLECAS,Colision,Despejado,Leve,2019-01-03 13:00:00,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA
4,2019S000132,VILLA DE VALLECAS,Colision,Despejado,Leve,2019-01-03 13:00:00,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA


In [ ]:
# encoding='cp850'
# encoding='cp1250'

# **
# encoding='latin_1'
# encoding='iso8859_15'

In [18]:
abr = pd.read_csv('tiempo/Anio2019/abr_meteo19.csv', encoding='latin_1',sep = ';')

In [20]:
abr.head(20)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,102,81,28079102_81_98,2019,4,1,1.90,V,...,1.45,V,0.90,V,0.97,V,1.68,V,1.12,V
1,28,79,102,81,28079102_81_98,2019,4,2,0.40,V,...,2.48,V,2.05,V,1.88,V,0.95,V,1.10,V
2,28,79,102,81,28079102_81_98,2019,4,3,1.18,V,...,3.95,V,3.53,V,3.40,V,3.42,V,2.58,V
3,28,79,102,81,28079102_81_98,2019,4,4,2.35,V,...,3.12,V,2.52,V,2.30,V,2.13,V,1.88,V
4,28,79,102,81,28079102_81_98,2019,4,5,3.02,V,...,2.88,V,1.43,V,0.93,V,2.18,V,2.58,V
5,28,79,102,81,28079102_81_98,2019,4,6,2.18,V,...,4.93,V,2.98,V,3.07,V,1.95,V,1.83,V
6,28,79,102,81,28079102_81_98,2019,4,7,2.77,V,...,1.73,V,1.48,V,0.73,V,0.37,V,0.75,V
7,28,79,102,81,28079102_81_98,2019,4,8,1.18,V,...,3.95,V,2.82,V,2.50,V,4.33,V,1.30,V
8,28,79,102,81,28079102_81_98,2019,4,9,1.22,V,...,3.48,V,3.92,V,3.15,V,2.98,V,3.67,V
9,28,79,102,81,28079102_81_98,2019,4,10,1.85,V,...,2.02,V,1.65,V,1.68,V,1.73,V,1.60,V
